In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
base_dir = r"data"


train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)
train_datagen=train_datagen.flow_from_directory(base_dir,target_size=(640,480),subset="training",batch_size=2)
test_datagen=test_datagen.flow_from_directory(base_dir,target_size=(640,480),subset="validation",batch_size=2)


Found 299 images belonging to 2 classes.
Found 32 images belonging to 2 classes.


In [13]:
import numpy as np
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D , MaxPooling2D , Dense, Dropout,Activation,Flatten 
from tensorflow.keras import layers,activations
from tensorflow.keras.applications import VGG19


In [4]:
from tensorflow.python.keras.applications.resnet import ResNet50 
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense

In [12]:
resenet = ResNet50(include_top=False,
                    input_shape=(224,244,3),
                    pooling="avg",classes=2,
                    weights="imagenet")
resenet.summary()

94773248/94765736 [==============================] - 16s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 244, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 250, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 122, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 122, 64) 256         conv1_conv[0][0]                 
_________________________

In [14]:
model = Sequential()
for layer in resenet.layers:
    layer.trainable = False

model.add(resenet)
model.add(Flatten())

model.add(Dense(512,activation="relu"))
model.add(Dense(2,activation="softmax"))

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 24,637,826
Trainable params: 1,050,114
Non-trainable params: 23,587,712
_________________________________________________________________


In [15]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer,loss=loss,metrics=['accuracy'])

In [16]:
result = model.fit(train_datagen,epochs=2,verbose=1,validation_data=test_datagen)

Epoch 1/2
150/150 [==============================] - 235s 2s/step - loss: 1.2939 - accuracy: 0.5753 - val_loss: 0.6738 - val_accuracy: 0.4375
Epoch 2/2
150/150 [==============================] - 241s 2s/step - loss: 0.5753 - accuracy: 0.6923 - val_loss: 0.5438 - val_accuracy: 0.6250
